# Fetch Data Challenge
You will be provided with a dataset of offers and some associated metadata around the retailers and brands that are sponsoring the offer.

You will also be provided with a dataset of some brands that we support on our platform, and the categories that those products belong
to.

# Acceptance Criteria
- If a user searches for a category (ex. diapers) the tool should return a list of offers that are relevant to that
category.
- If a user searches for a brand (ex. Huggies) the tool should return a list of offers that are relevant to that brand.
- If a user searches for a retailer (ex. Target) the tool should return a list of offers that are relevant to that retailer.
- The tool should also return the score that was used to measure the similarity of the text input with each offer

# What we seek below:
- Detailed responses to each problem, with a focus on the production pipeline surrounding the model.
- Identifies several useful techniques to approach eReceipt classification and entity extraction.
- Demonstrate a knowledge of recent innovations in NLP and a willingness to think about the problem in terms of software engineering rather than an academic exercise

## Data Inspection and EDA

In [1]:
# check current path
import os
import pandas as pd
pd.set_option('display.max_colwidth', None)


# # Load the Drive helper and mount
# from google.colab import drive
# # This will prompt for authorization.
# drive.mount('/content/drive/')
# path_gdrive = '/content/drive/MyDrive/Colab Datasets/Fetch'
# os.chdir(path_gdrive)
# print(os.getcwd())

In [2]:
path = "/Users/anthony/Projects/retailer_nlp_challenger/data"
os.chdir(path)

00-EDA.ipynb                    README.md
01-Train data cleanup.ipynb     brand_belong_category_dict.json
02-Data Converter.ipynb         config_transformer.cfg
03-Colab_spacy_train.ipynb      corpus
04-Search Processing.ipynb      data
10-spaCy test.ipynb             project.yml
11-Gradio Test.ipynb            src


## Categories.csv

In [3]:
import pandas as pd

df_categories = pd.read_csv('data/categories.csv')
df_categories

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness
...,...,...,...
113,0b039c0e-d33d-4356-b57b-83352d98623f,Frozen Turkey,Frozen Meat
114,3b79dd23-c298-4429-b9bf-ce5803b594eb,Frozen Chicken,Frozen Meat
115,7fbb4211-de07-4074-b359-aea21a7ad50c,Frozen Beef,Frozen Meat
116,a9ace557-fce3-4eec-9536-0e4b399987b7,Frozen Seafood,Frozen Meat


In [4]:
# how many categories? how many category id's?
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CATEGORY_ID           118 non-null    object
 1   PRODUCT_CATEGORY      118 non-null    object
 2   IS_CHILD_CATEGORY_TO  118 non-null    object
dtypes: object(3)
memory usage: 2.9+ KB


In [5]:
cat_features = df_categories.select_dtypes(include=['object']).columns.tolist()

In [6]:
for i in cat_features:
    print(i,len(df_categories[i].value_counts()))

CATEGORY_ID 118
PRODUCT_CATEGORY 118
IS_CHILD_CATEGORY_TO 23


In [7]:
# find if there're overlaps between categories and upper categories
all_categories = df_categories['PRODUCT_CATEGORY'].unique().tolist()
upper_categories = df_categories['IS_CHILD_CATEGORY_TO'].unique().tolist()

common_categories = [i for i in all_categories if i in upper_categories]
print(common_categories)
print(len(common_categories))

['Candy', 'Frozen', 'Dairy', 'Household Supplies', 'Puffed Snacks', 'Oral Care', 'Spirits', 'Pasta & Noodles', 'Mature']
9


In [8]:
upper_categories

['Pasta Sauce',
 'Pantry',
 'Health & Wellness',
 'Deli & Bakery',
 'Dairy',
 'Beverages',
 'Frozen',
 'Home & Garden',
 'Snacks',
 'Baby & Toddler',
 'Household Supplies',
 'Alcohol',
 'Oral Care',
 'Spirits',
 'Pasta & Noodles',
 'Puffed Snacks',
 'Meat & Seafood',
 'Beauty',
 'Mature',
 'Animals & Pet Supplies',
 'Sports Drinks & Enhanced Waters',
 'Frozen Meat',
 'Candy']

In [9]:
import re

def search_list(pattern, string_list):
    matches = []
    for string in string_list:
        if re.search(pattern, string, flags=re.IGNORECASE):
            matches.append(string)
    if len(matches) == 0:
        print("no matches")
        return None
    else:
        return matches

In [10]:
search_list(r'Alcohol', all_categories)

no matches


In [11]:
df_categories[df_categories["PRODUCT_CATEGORY"].eq('Wine')]

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
50,ca1c0f4d-3efc-4978-8357-69862996f416,Wine,Alcohol


In [12]:
df_categories[df_categories["IS_CHILD_CATEGORY_TO"].eq('Alcohol')]

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
29,a3ef4899-2bd2-4cac-bb31-a46ef1169c8c,Beer,Alcohol
36,6b770f8d-09b5-4491-8bbb-31ee38645b20,Malt Beverages,Alcohol
39,d5cba6a8-9ac7-40dd-9692-f9e890a48ca4,"Hard Seltzers, Sodas, Waters, Lemonades & Teas",Alcohol
40,b06d9098-f313-4ba8-88aa-a001db2759d8,Hard Ciders,Alcohol
50,ca1c0f4d-3efc-4978-8357-69862996f416,Wine,Alcohol
82,2def9983-9d56-4872-a5b9-7aa8bbc2331c,Spirits,Alcohol


For categories.csv, we know:
- There are 118 categories in total
- There are only 118-9=109 child categories

## retailer.csv

In [13]:
df_offers = pd.read_csv('data/offer_retailer.csv')
df_offers

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dillons Food Store",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at Amazon Storefront*",AMAZON,GATORADE
...,...,...,...
379,Spend $10 at KFC,KFC,KFC
380,Sargento Product,NaN,SARGENTO
381,Thomas'® Bagel Thins,NaN,THOMAS
382,Spend $270 at Pavilions,PAVILIONS,PAVILIONS


In [14]:
df_offers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   OFFER     384 non-null    object
 1   RETAILER  238 non-null    object
 2   BRAND     384 non-null    object
dtypes: object(3)
memory usage: 9.1+ KB


In [15]:
# notice a column with NaN; check null counts
df_offers.isnull().sum()

OFFER         0
RETAILER    146
BRAND         0
dtype: int64

In [16]:
# check categorical feature uniqueness
for i in df_offers.columns.tolist():
    print(i,len(df_offers[i].value_counts()))

OFFER 376
RETAILER 61
BRAND 144


In [17]:
# find if there're overlaps between retailers and brands
retailers = df_offers['RETAILER'].unique().tolist()
brands = df_offers['BRAND'].unique().tolist()

# what brands are also the retailers
[i for i in retailers if i in brands]

['SAMS CLUB',
 'ZAXBYS',
 'SUBWAY',
 'SHAWS',
 'ACME',
 'KFC',
 'CASEYS GENERAL STORE',
 'RANDALLS',
 'VONS',
 'FRESH THYME MARKET',
 'ALBERTSONS',
 'BJS WHOLESALE',
 'TOM THUMB',
 'SAFEWAY',
 'TGI FRIDAYS',
 'PAVILIONS',
 'STAR MARKET',
 'BLUE APRON',
 'MCALISTERS DELI',
 'COSTCO',
 'FARMER BOYS',
 'CHEWY',
 'DICKEYS BARBECUE PIT',
 'CVS',
 'BURGER KING']

In [18]:
# I call them b_retailer
# does such retailer provide different brand offers?
b_retailer = [i for i in retailers if i in brands]

# group by b_retailer and count the distinct brands they have
df_offers[df_offers['RETAILER'].isin(b_retailer)].groupby('RETAILER')['BRAND'].nunique()

RETAILER
ACME                    1
ALBERTSONS              1
BJS WHOLESALE           1
BLUE APRON              1
BURGER KING             1
CASEYS GENERAL STORE    1
CHEWY                   2
COSTCO                  1
CVS                     1
DICKEYS BARBECUE PIT    1
FARMER BOYS             1
FRESH THYME MARKET      1
KFC                     1
MCALISTERS DELI         1
PAVILIONS               1
RANDALLS                1
SAFEWAY                 1
SAMS CLUB               3
SHAWS                   1
STAR MARKET             1
SUBWAY                  1
TGI FRIDAYS             1
TOM THUMB               1
VONS                    1
ZAXBYS                  1
Name: BRAND, dtype: int64

In [19]:
df_offers[df_offers["RETAILER"].eq("SAMS CLUB")] #["BRAND"].value_counts()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
47,"Tyson Products, select varieties, spend $20 at Sam's Club",SAMS CLUB,BALL PARK FRANK
107,"George's Farmers Market Chicken Wings, at Sam's Club",SAMS CLUB,GEORGES FARMERS MARKET
273,"Spend $110 on a Full-Priced new Plus Membership and receive an ADDITIONAL 10,000 points",SAMS CLUB,SAMS CLUB


In [20]:
retailers2 = [i for i in retailers if isinstance(i, str)]
[i for i in retailers2 if ".com" in i.lower()]

['GALLO.COM', 'SIOBEAUTY.COM']

## Brand

In [21]:
df_brand = pd.read_csv('data/brand_category.csv')
df_brand

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844
...,...,...,...
9901,WIBBY BREWING,Beer,11
9902,LA FETE DU ROSE,Wine,11
9903,BIG ISLAND BREWHAUS,Beer,11
9904,BRIDGE LANE,Wine,11


In [22]:
df_brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9906 entries, 0 to 9905
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   BRAND                      9905 non-null   object
 1   BRAND_BELONGS_TO_CATEGORY  9906 non-null   object
 2   RECEIPTS                   9906 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 232.3+ KB


In [23]:
# why is one BRAND null?
df_brand[df_brand["BRAND"].isnull() == True]

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
6624,NaN,Beer,33


Clearly the brands should be cleaned because some brand names do not make sense. For example, you might have names like `3M` or `Forever21`, but no brand names are `21`.

In [24]:
grouped_df = df_brand.groupby('BRAND')['BRAND_BELONGS_TO_CATEGORY'].unique().reset_index()
print(grouped_df.shape)
grouped_df

(8521, 2)


,BRAND,BRAND_BELONGS_TO_CATEGORY
0,1,[Wine]
1,10 BARREL BREWING CO,"[Spirits, Beer, Fruit Juices]"
2,10 TORR,[Spirits]
3,100 NAILS RANCH,[Wine]
4,101 CIDER HOUSE,[Fruit Juices]
...,...,...
8516,ZYRTEC,[Medicines & Treatments]
8517,ZYWIEC,[Beer]
8518,Zagnut,[Candy]
8519,barkThins,[Candy]


In [25]:
# clean the rows based on BRAND: (1) NaN (2) only has numbers
df_brand = df_brand.dropna(subset=['BRAND'])
df_brand = df_brand[~df_brand['BRAND'].str.isnumeric()]
df_brand.shape

(9885, 3)

In [26]:
# check categorical feature uniqueness
for i in df_brand.columns.tolist():
    print(i,len(df_brand[i].value_counts()))

BRAND 8501
BRAND_BELONGS_TO_CATEGORY 118
RECEIPTS 2094


In [27]:
grouped_df = df_brand.groupby('BRAND')['BRAND_BELONGS_TO_CATEGORY'].unique().reset_index()
print(grouped_df.shape)
grouped_df

(8501, 2)


,BRAND,BRAND_BELONGS_TO_CATEGORY
0,10 BARREL BREWING CO,"[Spirits, Beer, Fruit Juices]"
1,10 TORR,[Spirits]
2,100 NAILS RANCH,[Wine]
3,101 CIDER HOUSE,[Fruit Juices]
4,1010 PREMIUM DRINKS,[Spirits]
...,...,...
8496,ZYRTEC,[Medicines & Treatments]
8497,ZYWIEC,[Beer]
8498,Zagnut,[Candy]
8499,barkThins,[Candy]


In [28]:
grouped_df.BRAND_BELONGS_TO_CATEGORY.apply(len).describe()

count    8501.000000
mean        1.162804
std         0.894340
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        29.000000
Name: BRAND_BELONGS_TO_CATEGORY, dtype: float64

So, one brand might correspond to multiple categories in a few cases

In [29]:
filtered_df = grouped_df[grouped_df['BRAND_BELONGS_TO_CATEGORY'].apply(len) > 1]
filtered_df

,BRAND,BRAND_BELONGS_TO_CATEGORY
0,10 BARREL BREWING CO,"[Spirits, Beer, Fruit Juices]"
47,4 HANDS BREWING CO,"[Beer, Malt Beverages]"
67,603 BREWERY,"[Malt Beverages, Beer]"
70,7 ELEVEN,"[Frozen Fruits, Cereal, Granola, & Toaster Pastries]"
106,A+D,"[Medicines & Treatments, Baby Bathing]"
...,...,...
8377,WRIGLEYS,"[Gum, Packaged Vegetables]"
8379,WRITE DUDES,"[Household Supplies, Makeup]"
8458,Z,"[Water, Spirits]"
8469,ZEVO,"[Household Supplies, Pest Control]"


One brand can belong to mutiple categories. And one category surely has mutiple brands.

## Create new table

In [30]:
df_offers = pd.read_csv('data/brand_offer_cleaned.csv')
df_offers.sample(5)

,BRAND,OFFER,idx
89,BACK TO THE ROOTS,"Back to the Roots Dry Plant Food, 5 pounds, at The Home Depot",130
222,CVS,Spend 10 at CVS,308
189,BEYOND MEAT,"Beyond Meat Plant-Based products, spend 20",267
238,BACK TO THE ROOTS,Back to the Roots Grow Kits at Walmart or The Home Depot,328
284,BACK TO THE ROOTS,"Back to the Roots Soils, select varieties, at Walmart",383


In [31]:
df_offers.shape

(285, 3)

In [32]:
# reload data
df_brand = pd.read_csv('data/brand_category.csv')
print(f'Brand original shape: {df_brand.shape}')

# clean the rows based on BRAND: (1) NaN (2) only has numbers
df_brand = df_brand.dropna(subset=['BRAND'])
df_brand = df_brand[~df_brand['BRAND'].str.isnumeric()]
print(f'Brand cleaned shape: {df_brand.shape}')

df_brand.sample(5)

Brand original shape: (9906, 3)
Brand cleaned shape: (9885, 3)


,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
9712,BIAGIO,Wine,11
6276,BEACHWOOD BREWING,Beer,39
1051,CIDERBOYS,Fruit Juices,1670
30,DAILYS,Wine,48829
3599,MIRINDA,Carbonated Soft Drinks,153


In [33]:
# find for each category, what brands does it have
grouped_df = df_brand.groupby('BRAND_BELONGS_TO_CATEGORY')['BRAND'].unique().reset_index()
print(grouped_df.shape)
grouped_df = grouped_df.rename(columns={'BRAND_BELONGS_TO_CATEGORY': 'CATEGORY'})
grouped_df.head()

(118, 2)


,CATEGORY,BRAND
0,Adult Incontinence,"[POISE, DEPEND, ALWAYS, WELLNESS, TENA, FIORE, ONE BY POISE]"
1,Alfredo & White Pasta Sauce,"[CLASSICO, PREGO, RAGU, RAO'S, KROGER, SIMPLE TRUTH, PUBLIX, HEB]"
2,Baby Bathing,"[BABY SOFT, PREDO, GERBER, ENFAMIL, EARTH'S BEST, HAPPY BABY, PAMPERS, SPROUT, MOTT'S, JOHNSONS, SHEA MOISTURE, AVEENO BABY, BABY DOVE, HAPPY TOT, PLUM ORGANICS, SIMILAC, THE HONEST CO., NANIT, SIMPLE TRUTH, MIKU, BEECH NUT, BABY SOFT EXPRESSIONS, ORGANICS HAPPY BABY, A+D, MUNCHKIN, VIVVI & BLOOM, ENFAGROW, HORIZON (WHITEWAVE FOODS), DR BROWN'S, GREENWISE, NATURE BABY CARE, NUTRAMIGEN, BABYGANICS, HOT-KID, CEREBELLY, BABY LOVE, AVEENO, SMART CARE, LUV N' CARE, LANSINOH, PIPETTE]"
3,Baby Health,"[GERBER, ENFAMIL, EARTH'S BEST, EVENFLO (EVENFLO FEEDING INC), AVENT]"
4,Baby Safety,[PAMPERS]


In [34]:
# write into a dict
brand_belong_category_dict = {row['CATEGORY']: row['BRAND'].tolist() for index, row in grouped_df.iterrows()}
brand_belong_category_dict

{'Adult Incontinence': ['POISE',
  'DEPEND',
  'ALWAYS',
  'WELLNESS',
  'TENA',
  'FIORE',
  'ONE BY POISE'],
 'Alfredo & White Pasta Sauce': ['CLASSICO',
  'PREGO',
  'RAGU',
  "RAO'S",
  'KROGER',
  'SIMPLE TRUTH',
  'PUBLIX',
  'HEB'],
 'Baby Bathing': ['BABY SOFT',
  'PREDO',
  'GERBER',
  'ENFAMIL',
  "EARTH'S BEST",
  'HAPPY BABY',
  'PAMPERS',
  'SPROUT',
  "MOTT'S",
  'JOHNSONS',
  'SHEA MOISTURE',
  'AVEENO BABY',
  'BABY DOVE',
  'HAPPY TOT',
  'PLUM ORGANICS',
  'SIMILAC',
  'THE HONEST CO.',
  'NANIT',
  'SIMPLE TRUTH',
  'MIKU',
  'BEECH NUT',
  'BABY SOFT EXPRESSIONS',
  'ORGANICS HAPPY BABY',
  'A+D',
  'MUNCHKIN',
  'VIVVI & BLOOM',
  'ENFAGROW',
  'HORIZON (WHITEWAVE FOODS)',
  "DR BROWN'S",
  'GREENWISE',
  'NATURE BABY CARE',
  'NUTRAMIGEN',
  'BABYGANICS',
  'HOT-KID',
  'CEREBELLY',
  'BABY LOVE',
  'AVEENO',
  'SMART CARE',
  "LUV N' CARE",
  'LANSINOH',
  'PIPETTE'],
 'Baby Health': ['GERBER',
  'ENFAMIL',
  "EARTH'S BEST",
  'EVENFLO (EVENFLO FEEDING INC)',
  '

In [35]:
# save the dict to json
import json
with open('data/brand_belong_category_dict.json', 'w') as fp:
    json.dump(brand_belong_category_dict, fp)

In [36]:
import pickle
# get brand names we have existing offers for
offered_brands = df_offers.BRAND.unique().tolist()

# # write them to csv 
# with open('data/offered_brands.pkl', 'wb') as f:
#     pickle.dump(offered_brands, f)